#### _Matin's Lab Rotation 3 - BCFO_ | [NEXT](00-1_build_strain.ipynb)
# 01. Find Genes to Knockout
* In this first part, we want to find which genes need to be knocked out from the model
* List of knockout genes provided by Yixin:

## Xylitol strain
### Knock outs
* focA, https://www.uniprot.org/uniprot/P0AC23
* pflB, https://www.uniprot.org/uniprot/P09373
* ldhA, https://www.uniprot.org/uniprot/P52643
* adhE, https://www.uniprot.org/uniprot/P0A9Q7
* xylAB, https://www.uniprot.org/uniprot/D1MDQ9 or https://www.uniprot.org/uniprot/P09099?
* frdA, https://www.uniprot.org/uniprot/P00363
* cyoB, https://www.uniprot.org/uniprot/P0ABI8
* appB, https://www.uniprot.org/uniprot/P26458

### Missing reaction(s)
* xylose + NADPH -> xylitol + NADP+ https://www.brenda-enzymes.org/enzyme.php?ecno=1.1.1.307
add this reactaion https://www.uniprot.org/uniprot/Q8X195

## Isobutyric acid strain
### Knock outs
* focA, https://www.uniprot.org/uniprot/P0AC23
* pflB, https://www.uniprot.org/uniprot/P09373
* aceEF, https://www.uniprot.org/uniprot/P0ACL9
* poxB, https://www.uniprot.org/uniprot/P07003
* tdcE, https://www.uniprot.org/uniprot/P42632
* pflDC, https://www.uniprot.org/uniprot/P32675 
* deoC, https://www.uniprot.org/uniprot/P0A6L0
* ydbK, https://www.uniprot.org/uniprot/P52647
* yqhD, https://www.uniprot.org/uniprot/Q46856
* araB, https://www.uniprot.org/uniprot/P08204
* araA, https://www.uniprot.org/uniprot/P08202
* xylAB, https://www.uniprot.org/uniprot/D1MDQ9


### Missing reactions(s): 
* isobutyraldehyde to isobutyric acid (forgot the enzyme/gene name)
Catalyzed by padA and has the following reaction (replace substrate and product):
* phenylacetaldehyde + NAD+ + H2O → phenylacetate + NADH + 2 H+
* Originally identified to oxidize isobutyraldehyde in this paper:
https://findit.dtu.dk/en/catalog/2482526722
https://www.uniprot.org/uniprot/P80668
http://bigg.ucsd.edu/models/iML1515/reactions/ALDD19xr
* Also add this gene:
kivD - https://www.uniprot.org/uniprot/D2BR82
https://biocyc.org/META/NEW-IMAGE?object=RXN-7643

# Write the knockout list in a table
Find the table in:
* [data_ko_xylitol_strain.txt](data/data_ko_xylitol_strain.txt)
* [data_ko_isobutyric-acid_strain](data/data_ko_isobutyric-acid_strain.txt)

# Load Library

In [1]:
from bioservices import UniProt
import io
import pandas as pd
import numpy as np

from cameo import models
from cameo import load_model

service = UniProt()

/home/matinnu/anaconda3/envs/lab_rotation_3/lib/python3.7/site-packages/sympy/__init__.py:676: SymPyDeprecationWarning: 

importing sympy.core.singleton with 'from sympy import *' has been
deprecated since SymPy 1.6. Use import sympy.core.singleton instead.
See https://github.com/sympy/sympy/issues/18245 for more info.

  deprecated_since_version="1.6").warn()


In [2]:
# get value from uniprot

def get_uniprot(path, strain_name):
    df_input = pd.read_csv(path, header = None,
                     skiprows = 0,
                     sep = ',',
                     names = [
                         'gene',
                         'uniprot',
                         'strain'])
    df_input['strain'] = strain_name
    df = pd.DataFrame(columns=["Entry", "Entry name", "Gene names  (primary )", "Gene names  (synonym )", "Protein names","Gene ontology (molecular function)","Catalytic activity"])
    for i in df_input.uniprot:
        uniprot_id = i.split('/')[-1]
        # Make a query string
        query = "accession:"+uniprot_id
        # Define a list of columns we want to retrive
        columnlist = "id,entry name,genes(PREFERRED),genes(ALTERNATIVE),protein names,go(molecular function),comment(CATALYTIC ACTIVITY)"
        # Run the remote search
        result = service.search(query, frmt="tab", columns=columnlist)
        df_add = (pd.read_table(io.StringIO(result)))
        df = pd.concat([df, df_add])
    df = df.reset_index(drop=True)
    df_input = pd.merge(df_input, df, left_index=True, right_index=True)
    return df_input

def check_reaction(df):
    reaction_data = []
    for counter, i in enumerate(df['Catalytic activity']):
        try:
            reaction = i.split(';')[0].replace('CATALYTIC ACTIVITY: Reaction=','')
            print(df.gene[counter]+':', reaction)
            reaction_data.append(reaction)
        except:
            print(df.gene[counter]+':', 'reaction not available')
            reaction_data.append(np.nan)
    df = df.assign(Reaction = reaction_data)
    return

# Find knockouts for xylitol strain

In [3]:
path = 'data/data_ko_xylitol_strain.txt'
strain_name = 'xylytol_mutant'
df_xylitol = get_uniprot(path, strain_name)
df_xylitol

,gene,uniprot,strain,Entry,Entry name,Gene names (primary ),Gene names (synonym ),Protein names,Gene ontology (molecular function),Catalytic activity
0,focA,https://www.uniprot.org/uniprot/P0AC23,xylytol_mutant,P0AC23,FOCA_ECOLI,focA,ycaE,Probable formate transporter 1 (Formate channe...,formate transmembrane transporter activity [GO...,NaN
1,pflB,https://www.uniprot.org/uniprot/P09373,xylytol_mutant,P09373,PFLB_ECOLI,pflB,pfl,Formate acetyltransferase 1 (EC 2.3.1.54) (Pyr...,formate C-acetyltransferase activity [GO:0008861],CATALYTIC ACTIVITY: Reaction=acetyl-CoA + form...
2,ldhA,https://www.uniprot.org/uniprot/P52643,xylytol_mutant,P52643,LDHD_ECOLI,ldhA,hslI htpH,D-lactate dehydrogenase (D-LDH) (EC 1.1.1.28) ...,D-lactate dehydrogenase activity [GO:0008720];...,CATALYTIC ACTIVITY: Reaction=(R)-lactate + NAD...
3,adhE,https://www.uniprot.org/uniprot/P0A9Q7,xylytol_mutant,P0A9Q7,ADHE_ECOLI,adhE,ana,Aldehyde-alcohol dehydrogenase [Includes: Alco...,acetaldehyde dehydrogenase (acetylating) activ...,CATALYTIC ACTIVITY: Reaction=a primary alcohol...
4,xylAB,https://www.uniprot.org/uniprot/P09099,xylytol_mutant,P09099,XYLB_ECOLI,xylB,NaN,Xylulose kinase (XK) (Xylulokinase) (EC 2.7.1....,ATP binding [GO:0005524]; xylulokinase activit...,CATALYTIC ACTIVITY: Reaction=ATP + D-xylulose ...
5,frdA,https://www.uniprot.org/uniprot/P00363,xylytol_mutant,P00363,FRDA_ECOLI,frdA,NaN,Fumarate reductase flavoprotein subunit (EC 1....,electron transfer activity [GO:0009055]; FAD b...,CATALYTIC ACTIVITY: Reaction=a quinone + succi...
6,cyoB,https://www.uniprot.org/uniprot/P0ABI8,xylytol_mutant,P0ABI8,CYOB_ECOLI,cyoB,NaN,Cytochrome bo(3) ubiquinol oxidase subunit 1 (...,copper ion binding [GO:0005507]; cytochrome bo...,CATALYTIC ACTIVITY: Reaction=2 a ubiquinol + n...
7,appB,https://www.uniprot.org/uniprot/P26458,xylytol_mutant,P26458,APPB_ECOLI,appB,cbdB cyxB,Cytochrome bd-II ubiquinol oxidase subunit 2 (...,electron transfer activity [GO:0009055]; metal...,CATALYTIC ACTIVITY: Reaction=2 a ubiquinol + n...


In [4]:
check_reaction(df_xylitol)

focA: reaction not available
pflB: acetyl-CoA + formate = CoA + pyruvate
ldhA: (R)-lactate + NAD(+) = H(+) + NADH + pyruvate
adhE: a primary alcohol + NAD(+) = an aldehyde + H(+) + NADH
xylAB: ATP + D-xylulose = ADP + D-xylulose 5-phosphate + H(+)
frdA: a quinone + succinate = a quinol + fumarate
cyoB: 2 a ubiquinol + n H(+)(in) + O2 = 2 a ubiquinone + n H(+)(out) + 2 H2O
appB: 2 a ubiquinol + n H(+)(in) + O2 = 2 a ubiquinone + n H(+)(out) + 2 H2O


# Find knockouts for isobutyric acid strain

In [5]:
path = 'data/data_ko_isobutyric-acid_strain.txt'
strain_name = 'isobutyric_mutant'
df_isobutyric = get_uniprot(path, strain_name)
df_isobutyric

,gene,uniprot,strain,Entry,Entry name,Gene names (primary ),Gene names (synonym ),Protein names,Gene ontology (molecular function),Catalytic activity
0,focA,https://www.uniprot.org/uniprot/P0AC23,isobutyric_mutant,P0AC23,FOCA_ECOLI,focA,ycaE,Probable formate transporter 1 (Formate channe...,formate transmembrane transporter activity [GO...,NaN
1,pflB,https://www.uniprot.org/uniprot/P09373,isobutyric_mutant,P09373,PFLB_ECOLI,pflB,pfl,Formate acetyltransferase 1 (EC 2.3.1.54) (Pyr...,formate C-acetyltransferase activity [GO:0008861],CATALYTIC ACTIVITY: Reaction=acetyl-CoA + form...
2,aceEF,https://www.uniprot.org/uniprot/P0ACL9,isobutyric_mutant,P0ACL9,PDHR_ECOLI,pdhR,aceC genA yacB,Pyruvate dehydrogenase complex repressor,DNA binding [GO:0003677]; DNA-binding transcri...,NaN
3,poxB,https://www.uniprot.org/uniprot/P07003,isobutyric_mutant,P07003,POXB_ECOLI,poxB,NaN,Pyruvate dehydrogenase [ubiquinone] (EC 1.2.5....,flavin adenine dinucleotide binding [GO:005066...,CATALYTIC ACTIVITY: Reaction=a ubiquinone + H2...
4,tdcE,https://www.uniprot.org/uniprot/P42632,isobutyric_mutant,P42632,TDCE_ECOLI,tdcE,yhaS,PFL-like enzyme TdcE (Keto-acid formate acetyl...,2-ketobutyrate formate-lyase activity [GO:0043...,CATALYTIC ACTIVITY: Reaction=2-oxobutanoate + ...
5,pflDC,https://www.uniprot.org/uniprot/P32675,isobutyric_mutant,P32675,PFLC_ECOLI,pflC,yijM,Pyruvate formate-lyase 2-activating enzyme (EC...,[formate-C-acetyltransferase]-activating enzym...,CATALYTIC ACTIVITY: Reaction=glycyl-[formate C...
6,deoC,https://www.uniprot.org/uniprot/P0A6L0,isobutyric_mutant,P0A6L0,DEOC_ECOLI,deoC,dra thyR,Deoxyribose-phosphate aldolase (DERA) (EC 4.1....,deoxyribose-phosphate aldolase activity [GO:00...,CATALYTIC ACTIVITY: Reaction=2-deoxy-D-ribose ...
7,ydbK,https://www.uniprot.org/uniprot/P52647,isobutyric_mutant,P52647,NIFJ_ECOLI,ydbK,NaN,Probable pyruvate-flavodoxin oxidoreductase (E...,"4 iron, 4 sulfur cluster binding [GO:0051539];...",CATALYTIC ACTIVITY: Reaction=CoA + 2 H(+) + ox...
8,yqhD,https://www.uniprot.org/uniprot/Q46856,isobutyric_mutant,Q46856,YQHD_ECOLI,yqhD,NaN,Alcohol dehydrogenase YqhD (EC 1.1.1.-),alcohol dehydrogenase (NADP+) activity [GO:000...,CATALYTIC ACTIVITY: Reaction=a primary alcohol...
9,araB,https://www.uniprot.org/uniprot/P08204,isobutyric_mutant,P08204,ARAB_ECOLI,araB,NaN,Ribulokinase (EC 2.7.1.16),ATP binding [GO:0005524]; phosphotransferase a...,CATALYTIC ACTIVITY: Reaction=ATP + D-ribulose ...


In [6]:
check_reaction(df_isobutyric)

focA: reaction not available
pflB: acetyl-CoA + formate = CoA + pyruvate
aceEF: reaction not available
poxB: a ubiquinone + H2O + pyruvate = a ubiquinol + acetate + CO2
tdcE: 2-oxobutanoate + CoA = formate + propanoyl-CoA
pflDC: glycyl-[formate C-acetyltransferase] + reduced [flavodoxin] + S-adenosyl-L-methionine = 5'-deoxyadenosine + glycin-2-yl radical-[formate C-acetyltransferase] + H(+) + L-methionine + semiquinone [flavodoxin]
deoC: 2-deoxy-D-ribose 5-phosphate = acetaldehyde + D-glyceraldehyde 3-phosphate
ydbK: CoA + 2 H(+) + oxidized [flavodoxin] + pyruvate = acetyl-CoA + CO2 + reduced [flavodoxin]
yqhD: a primary alcohol + NADP(+) = an aldehyde + H(+) + NADPH
araB: ATP + D-ribulose = ADP + D-ribulose 5-phosphate + H(+)
araA: L-arabinose = L-ribulose
xylAB: ATP + D-xylulose = ADP + D-xylulose 5-phosphate + H(+)


# Check if genes are in the model
## Which model should we use?
Here we get the model from BIGG and use E coli with the most reactions available

In [7]:
bigg_list = models.index_models_bigg()

In [8]:
# Find GEMS that starts with "Eschericia coli" and ends with "MG1655"
e_coli_mg1655 = bigg_list[bigg_list.organism.str.contains("^Escherichia coli.*MG1655$")]
e_coli_mg1655

,bigg_id,gene_count,reaction_count,organism,metabolite_count
0,e_coli_core,137,95,Escherichia coli str. K-12 substr. MG1655,72
2,iAF1260,1261,2382,Escherichia coli str. K-12 substr. MG1655,1668
3,iAF1260b,1261,2388,Escherichia coli str. K-12 substr. MG1655,1668
71,iJO1366,1367,2583,Escherichia coli str. K-12 substr. MG1655,1805
72,iJR904,904,1075,Escherichia coli str. K-12 substr. MG1655,761
76,iML1515,1516,2712,Escherichia coli str. K-12 substr. MG1655,1877


In [9]:
model_id = e_coli_mg1655.iloc[e_coli_mg1655.reaction_count.argmax()].bigg_id
model_id

'iML1515'

## Load Model

In [10]:
model = load_model(model_id)
model

Using license file /home/matinnu/gurobi.lic
Academic license - for non-commercial use only


Name,iML1515
Memory address,0x07f9f193bd790
Number of metabolites,1877
Number of reactions,2712
Number of groups,0
Objective expression,0.0 + 1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [11]:
# get all available genes and their index
gene_list = []
for counter, i in enumerate(model.genes):
    try:
        data = [str(i), counter, i.annotation['uniprot'][0]]
        gene_list.append(data)
    except:
        data = [str(i), counter, '']
        gene_list.append(data)
df_gene = pd.DataFrame(gene_list, columns=['id_bigg',model_id+'_idx','Entry'])
df_gene

,id_bigg,iML1515_idx,Entry
0,b2551,0,P0A825
1,b0870,1,P75823
2,b3368,2,P0AEA8
3,b2436,3,P36553
4,b0008,4,P0A870
...,...,...,...
1511,b3924,1511,P28861
1512,b4209,1512,P69506
1513,b1387,1513,P77455
1514,b1393,1514,P76082


## Find if the genes we want to knock out is in the model
### For Xylitol producing strain

In [12]:
df_xylitol = df_xylitol.join(df_gene.set_index('Entry'), on='Entry')
df_xylitol

,gene,uniprot,strain,Entry,Entry name,Gene names (primary ),Gene names (synonym ),Protein names,Gene ontology (molecular function),Catalytic activity,id_bigg,iML1515_idx
0,focA,https://www.uniprot.org/uniprot/P0AC23,xylytol_mutant,P0AC23,FOCA_ECOLI,focA,ycaE,Probable formate transporter 1 (Formate channe...,formate transmembrane transporter activity [GO...,NaN,b0904,1023
1,pflB,https://www.uniprot.org/uniprot/P09373,xylytol_mutant,P09373,PFLB_ECOLI,pflB,pfl,Formate acetyltransferase 1 (EC 2.3.1.54) (Pyr...,formate C-acetyltransferase activity [GO:0008861],CATALYTIC ACTIVITY: Reaction=acetyl-CoA + form...,b0903,162
2,ldhA,https://www.uniprot.org/uniprot/P52643,xylytol_mutant,P52643,LDHD_ECOLI,ldhA,hslI htpH,D-lactate dehydrogenase (D-LDH) (EC 1.1.1.28) ...,D-lactate dehydrogenase activity [GO:0008720];...,CATALYTIC ACTIVITY: Reaction=(R)-lactate + NAD...,b1380,1135
3,adhE,https://www.uniprot.org/uniprot/P0A9Q7,xylytol_mutant,P0A9Q7,ADHE_ECOLI,adhE,ana,Aldehyde-alcohol dehydrogenase [Includes: Alco...,acetaldehyde dehydrogenase (acetylating) activ...,CATALYTIC ACTIVITY: Reaction=a primary alcohol...,b1241,100
4,xylAB,https://www.uniprot.org/uniprot/P09099,xylytol_mutant,P09099,XYLB_ECOLI,xylB,NaN,Xylulose kinase (XK) (Xylulokinase) (EC 2.7.1....,ATP binding [GO:0005524]; xylulokinase activit...,CATALYTIC ACTIVITY: Reaction=ATP + D-xylulose ...,b3564,129
5,frdA,https://www.uniprot.org/uniprot/P00363,xylytol_mutant,P00363,FRDA_ECOLI,frdA,NaN,Fumarate reductase flavoprotein subunit (EC 1....,electron transfer activity [GO:0009055]; FAD b...,CATALYTIC ACTIVITY: Reaction=a quinone + succi...,b4154,171
6,cyoB,https://www.uniprot.org/uniprot/P0ABI8,xylytol_mutant,P0ABI8,CYOB_ECOLI,cyoB,NaN,Cytochrome bo(3) ubiquinol oxidase subunit 1 (...,copper ion binding [GO:0005507]; cytochrome bo...,CATALYTIC ACTIVITY: Reaction=2 a ubiquinol + n...,b0431,1216
7,appB,https://www.uniprot.org/uniprot/P26458,xylytol_mutant,P26458,APPB_ECOLI,appB,cbdB cyxB,Cytochrome bd-II ubiquinol oxidase subunit 2 (...,electron transfer activity [GO:0009055]; metal...,CATALYTIC ACTIVITY: Reaction=2 a ubiquinol + n...,b0979,799


In [13]:
#export the result
df_xylitol.to_csv('data/df_xylitol_knockouts.csv')

### For isobutyric acid producing strain

In [14]:
df_isobutyric = df_isobutyric.join(df_gene.set_index('Entry'), on='Entry')
df_isobutyric

,gene,uniprot,strain,Entry,Entry name,Gene names (primary ),Gene names (synonym ),Protein names,Gene ontology (molecular function),Catalytic activity,id_bigg,iML1515_idx
0,focA,https://www.uniprot.org/uniprot/P0AC23,isobutyric_mutant,P0AC23,FOCA_ECOLI,focA,ycaE,Probable formate transporter 1 (Formate channe...,formate transmembrane transporter activity [GO...,NaN,b0904,1023.0
1,pflB,https://www.uniprot.org/uniprot/P09373,isobutyric_mutant,P09373,PFLB_ECOLI,pflB,pfl,Formate acetyltransferase 1 (EC 2.3.1.54) (Pyr...,formate C-acetyltransferase activity [GO:0008861],CATALYTIC ACTIVITY: Reaction=acetyl-CoA + form...,b0903,162.0
2,aceEF,https://www.uniprot.org/uniprot/P0ACL9,isobutyric_mutant,P0ACL9,PDHR_ECOLI,pdhR,aceC genA yacB,Pyruvate dehydrogenase complex repressor,DNA binding [GO:0003677]; DNA-binding transcri...,NaN,NaN,NaN
3,poxB,https://www.uniprot.org/uniprot/P07003,isobutyric_mutant,P07003,POXB_ECOLI,poxB,NaN,Pyruvate dehydrogenase [ubiquinone] (EC 1.2.5....,flavin adenine dinucleotide binding [GO:005066...,CATALYTIC ACTIVITY: Reaction=a ubiquinone + H2...,b0871,175.0
4,tdcE,https://www.uniprot.org/uniprot/P42632,isobutyric_mutant,P42632,TDCE_ECOLI,tdcE,yhaS,PFL-like enzyme TdcE (Keto-acid formate acetyl...,2-ketobutyrate formate-lyase activity [GO:0043...,CATALYTIC ACTIVITY: Reaction=2-oxobutanoate + ...,b3114,147.0
5,pflDC,https://www.uniprot.org/uniprot/P32675,isobutyric_mutant,P32675,PFLC_ECOLI,pflC,yijM,Pyruvate formate-lyase 2-activating enzyme (EC...,[formate-C-acetyltransferase]-activating enzym...,CATALYTIC ACTIVITY: Reaction=glycyl-[formate C...,b3952,154.0
6,deoC,https://www.uniprot.org/uniprot/P0A6L0,isobutyric_mutant,P0A6L0,DEOC_ECOLI,deoC,dra thyR,Deoxyribose-phosphate aldolase (DERA) (EC 4.1....,deoxyribose-phosphate aldolase activity [GO:00...,CATALYTIC ACTIVITY: Reaction=2-deoxy-D-ribose ...,b4381,678.0
7,ydbK,https://www.uniprot.org/uniprot/P52647,isobutyric_mutant,P52647,NIFJ_ECOLI,ydbK,NaN,Probable pyruvate-flavodoxin oxidoreductase (E...,"4 iron, 4 sulfur cluster binding [GO:0051539];...",CATALYTIC ACTIVITY: Reaction=CoA + 2 H(+) + ox...,b1378,1428.0
8,yqhD,https://www.uniprot.org/uniprot/Q46856,isobutyric_mutant,Q46856,YQHD_ECOLI,yqhD,NaN,Alcohol dehydrogenase YqhD (EC 1.1.1.-),alcohol dehydrogenase (NADP+) activity [GO:000...,CATALYTIC ACTIVITY: Reaction=a primary alcohol...,b3011,77.0
9,araB,https://www.uniprot.org/uniprot/P08204,isobutyric_mutant,P08204,ARAB_ECOLI,araB,NaN,Ribulokinase (EC 2.7.1.16),ATP binding [GO:0005524]; phosphotransferase a...,CATALYTIC ACTIVITY: Reaction=ATP + D-ribulose ...,b0063,131.0


In [15]:
#export the result
df_isobutyric.to_csv('data/df_isobutyric_knockouts.csv')

# [CLICK FOR NEXT STEP](00-1_build_strain.ipynb)